In [1]:
import shutil
import os
import random

original_dir = r"C:\Users\betti\Downloads\Mini_project_Image\Dataset_images\Image_Processing\Original"
pixelated_dir = r"C:\Users\betti\Downloads\Mini_project_Image\Dataset_images\Image_Processing\Pixelated"

test_folder = r"C:\Users\betti\Downloads\Mini_project_Image\Dataset_images\Image_Processing\Working\test"
os.makedirs(test_folder, exist_ok=True)

def copy_images(source_dir, destination_dir, num_images):
    images = os.listdir(source_dir)
    selected_images = random.sample(images, num_images)
    for image_name in selected_images:
        source_path = os.path.join(source_dir, image_name)
        destination_path = os.path.join(destination_dir, image_name)
        shutil.copyfile(source_path, destination_path)

num_images_per_category = 15  

copy_images(original_dir, test_folder, num_images_per_category)

copy_images(pixelated_dir, test_folder, num_images_per_category)

print(f"{num_images_per_category} original and {num_images_per_category} pixelated images copied to the test folder.")


15 original and 15 pixelated images copied to the test folder.


In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image_dataset_from_directory
import numpy as np
import os

# Load model
model_path = r"C:\Users\betti\Downloads\Mini_project_Image\pixel_classification_model.h5"
model = load_model(model_path)

# Test dataset directory
test_folder = r"C:\Users\betti\Downloads\Mini_project_Image\Dataset_images\Image_Processing\Working\test"

# Load dataset efficiently
batch_size = 32
img_height = 224
img_width = 224

test_ds = image_dataset_from_directory(
    test_folder,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False
)

# Normalize images
normalization_layer = tf.keras.layers.Rescaling(1./255)
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

# Predict batch-wise
predictions = model.predict(test_ds)
predictions = predictions.flatten()  # Convert to 1D array

# Assign class labels
threshold = 0.8
predicted_labels = ["Pixelated" if pred >= threshold else "High resolution" for pred in predictions]

print(predicted_labels)


Found 3560 files belonging to 2 classes.
112/112 ━━━━━━━━━━━━━━━━━━━━ 62s 531ms/step
['High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'High resolution', 'H

In [19]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image_dataset_from_directory
import numpy as np
import os
from sklearn.metrics import classification_report

# Load model
model_path = r"C:\Users\betti\Downloads\Mini_project_Image\pixel_classification_model.h5"
model = load_model(model_path)

# Test dataset directory
test_folder = r"C:\Users\betti\Downloads\Mini_project_Image\Dataset_images\Image_Processing\Working\test"

# Load dataset efficiently
batch_size = 32
img_height = 224
img_width = 224

test_ds = image_dataset_from_directory(
    test_folder,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False
)

# Get true labels
true_labels = np.concatenate([y.numpy() for _, y in test_ds])

# Normalize images
normalization_layer = tf.keras.layers.Rescaling(1./255)
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

# Predict batch-wise
predictions = model.predict(test_ds)
predictions = predictions.flatten()  # Convert to 1D array

# Assign class labels
threshold = 0.8
predicted_labels = [1 if pred >= threshold else 0 for pred in predictions]  # 1: Pixelated, 0: High resolution

# Generate classification report
report = classification_report(true_labels, predicted_labels, target_names=["High resolution", "Pixelated"])
print(report)


Found 3560 files belonging to 2 classes.
112/112 ━━━━━━━━━━━━━━━━━━━━ 64s 544ms/step
                 precision    recall  f1-score   support

High resolution       0.93      0.99      0.96      1780
      Pixelated       0.99      0.92      0.96      1780

       accuracy                           0.96      3560
      macro avg       0.96      0.96      0.96      3560
   weighted avg       0.96      0.96      0.96      3560



In [13]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image_dataset_from_directory
import numpy as np
import os
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Load model
model_path = r"C:\Users\betti\Downloads\Mini_project_Image\pixel_classification_model.h5"
model = load_model(model_path)

# Test dataset directory
test_folder = r"C:\Users\betti\Downloads\Mini_project_Image\Dataset_images\Image_Processing\Working\test"

# Load dataset efficiently
batch_size = 32
img_height = 224
img_width = 224

test_ds = image_dataset_from_directory(
    test_folder,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False
)

# Get true labels
true_labels = np.concatenate([y.numpy() for _, y in test_ds])

# Normalize images
normalization_layer = tf.keras.layers.Rescaling(1./255)
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

# Predict batch-wise
predictions = model.predict(test_ds)
predictions = predictions.flatten()  # Convert to 1D array

# Assign class labels based on threshold
threshold = 0.8
predicted_labels = [1 if pred >= threshold else 0 for pred in predictions]  # 1: Pixelated, 0: High resolution

# Generate confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Print confusion matrix as text
print("Confusion Matrix:")
print(cm)

# Plot confusion matrix nicely with seaborn heatmap
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=["High resolution", "Pixelated"],
            yticklabels=["High resolution", "Pixelated"])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


ImportError: generic_type: cannot initialize type "StatusCode": an object with that name is already defined

In [27]:
import time
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import os

model_path = r"C:\Users\betti\Downloads\Mini_project_Image\pixel_classification_model.h5"
model = load_model(model_path)

def preprocess_image(image_path, img_height=224, img_width=224):
    img = image.load_img(image_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  
    img_array = img_array / 255.0  
    return img_array


def classify_image(image_path, model, threshold=0.8):
    img = preprocess_image(image_path)
    start_time = time.time()
    prediction = model.predict(img)
    end_time = time.time()
    fps = 1 / (end_time - start_time)
    
    if prediction >= threshold:
        return "Pixelated", fps
    else:
        return "High resolution", fps

image_path = r"C:\Users\betti\Downloads\img_2.jpg"
result, fps = classify_image(image_path, model)
print(f"Image classified as {result} with FPS: {fps:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Image classified as High resolution with FPS: 0.44
